# Tutorial: Learn the basics of Druid SQL

<!--
  ~ Licensed to the Apache Software Foundation (ASF) under one
  ~ or more contributor license agreements.  See the NOTICE file
  ~ distributed with this work for additional information
  ~ regarding copyright ownership.  The ASF licenses this file
  ~ to you under the Apache License, Version 2.0 (the
  ~ "License"); you may not use this file except in compliance
  ~ with the License.  You may obtain a copy of the License at
  ~
  ~   http://www.apache.org/licenses/LICENSE-2.0
  ~
  ~ Unless required by applicable law or agreed to in writing,
  ~ software distributed under the License is distributed on an
  ~ "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
  ~ KIND, either express or implied.  See the License for the
  ~ specific language governing permissions and limitations
  ~ under the License.
  -->
  
Druid SQL is a Structured Query Language (SQL) dialect that enables you to query datasources in Apache Druid using SQL statements.
SQL and Druid SQL use similar syntax, with some notable differences.
Not all SQL functions are supported in Druid SQL. Instead, Druid includes Druid-specific SQL functions for optimized query performance.

This interactive tutorial introduces you to the unique aspects of Druid SQL.

## Prerequisites

Make sure that you meet the requirements outlined in the README.md file of the [apache/druid repo](https://github.com/apache/druid/tree/master/examples/quickstart/jupyter-notebooks/).
Specifically, you need the following:
- Knowledge of SQL
- Python3
- The `requests` package for Python
- JupyterLab (recommended) or Jupyter Notebook running on a non-default port. Druid and Jupyter both default to port `8888`, so you need to start Jupyter on a different port. 
- An available Druid instance. This tutorial uses the `micro-quickstart` configuration described in the [Druid quickstart](https://druid.apache.org/docs/latest/tutorials/index.html), so no authentication or authorization is required unless explicitly mentioned. If you haven’t already, download Druid and start Druid services as described in the quickstart.

## Prepare your environment

This section contains the steps required to prepare your environment to follow along with this tutorial.

Start by running the following cell. It imports the required Python packages and defines a variable for the Druid host.

In [ ]:
import requests
import json

# druid_host is the hostname and port for your Druid deployment. 
druid_host = "http://localhost:8888"
dataSourceName = "wikipedia-sql-tutorial"
print(druid_host)

In the rest of the tutorial, the `endpoint`, `http_method`, and `payload` variables are updated to accomplish different tasks.

Run the following cell to ingest data from an external source into a table named `wikipedia-sql-tutorial` using the [multi-stage query (MSQ) task engine](https://druid.apache.org/docs/latest/multi-stage-query/index.html).

In [ ]:
endpoint = "/druid/v2/sql/task"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
    "query": "INSERT INTO \"wikipedia-sql-tutorial\" SELECT TIME_PARSE(\"timestamp\") AS __time, * FROM TABLE( EXTERN( '{\"type\": \"http\", \"uris\": [\"https://druid.apache.org/data/wikipedia.json.gz\"]}','{\"type\": \"json\"}', '[{\"name\": \"added\", \"type\": \"long\"}, {\"name\": \"channel\", \"type\": \"string\"}, \  {\"name\": \"cityName\", \"type\": \"string\"}, {\"name\": \"comment\", \"type\": \"string\"}, {\"name\": \"commentLength\", \"type\": \"long\"}, {\"name\": \"countryIsoCode\", \"type\": \"string\"}, {\"name\": \"countryName\", \"type\": \"string\"}, {\"name\": \"deleted\", \"type\": \"long\"},{\"name\": \"delta\", \"type\": \"long\"}, {\"name\": \"deltaBucket\", \"type\": \"string\"}, {\"name\": \"diffUrl\", \"type\": \"string\"}, {\"name\": \"flags\", \"type\": \"string\"}, {\"name\": \"isAnonymous\", \"type\": \"string\"}, {\"name\": \"isMinor\", \"type\": \"string\"},  {\"name\": \"isNew\", \"type\": \"string\"}, {\"name\": \"isRobot\", \"type\": \"string\"}, {\"name\": \"isUnpatrolled\", \"type\": \"string\"}, {\"name\": \"metroCode\", \"type\": \"string\"}, {\"name\": \"namespace\", \"type\": \"string\"}, {\"name\": \"page\", \"type\": \"string\"}, {\"name\": \"regionIsoCode\", \"type\": \"string\"}, {\"name\": \"regionName\", \"type\": \"string\"},  {\"name\": \"timestamp\", \"type\": \"string\"}, {\"name\": \"user\", \"type\": \"string\"}]')) PARTITIONED BY DAY",
  "context": {
    "maxNumTasks": 3
  }
})

headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
ingestiion_taskId_response = response
ingestion_taskId = json.loads(response.text)['taskId']
print(ingestion_taskId + f"\nInserting data into the table named {dataSourceName}.")

To check the status of your ingestion task, run the following cell. 
When the value of the `status` property changes to `SUCCESS`, you can query your data.

In [ ]:
endpoint = f"/druid/indexer/v1/task/{ingestion_taskId}/status"
print(druid_host+endpoint)
http_method = "GET"

payload = {}
headers = {}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

## Datasources

Druid supports a variety of datasources, with the table datasource being the most common. In Druid documentation, the word "datasource" often implicitly refers to the table datasource.
The [Datasources](https://druid.apache.org/docs/latest/querying/datasource.html) topic provides a comprehensive overview of datasources supported by Druid SQL.

In Druid SQL, table datasources reside in the `druid` schema. This is the default schema, so table datasources can be referenced as either `druid.dataSourceName` or `dataSourceName`.

For example, run the next cell to return the rows of the column named `channel` from the `wikipedia-sql-tutorial` table. Because this tutorial is running in Jupyter, make sure to limit the size of your query results for display purposes using the SQL LIMIT clause.

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT \"channel\" FROM druid.wikipedia-sql-tutorial LIMIT 7"})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

## Data types

Druid maps SQL data types onto native types at query runtime.
The following native types are supported for Druid columns:

* STRING: UTF-8 encoded strings and string arrays
* LONG: 64-bit signed int
* FLOAT: 32-bit float
* DOUBLE: 64-bit float
* COMPLEX: represents non-standard data types, such as nested JSON, hyperUnique and approxHistogram aggregators, and DataSketches aggregators

Druid exposes table and column metadata through [INFORMATION_SCHEMA](https://druid.apache.org/docs/latest/querying/sql-metadata-tables.html#information-schema) tables. Run the following query to retrieve metadata for the `wikipedia-sql-tutorial` datasource. In the response body, each JSON object correlates to a column in the table.
Check the objects' `DATA_TYPE` property for SQL data types. You should see TIMESTAMP, BIGINT, and VARCHAR SQL data types. 

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE \"TABLE_SCHEMA\" = 'druid' AND \"TABLE_NAME\" = 'wikipedia-sql-tutorial' LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

Druid natively interprets VARCHAR as STRING and BIGINT and TIMESTAMP SQL data types as LONG. For reference on how SQL data types map onto Druid native types, see [Standard types](https://druid.apache.org/docs/latest/querying/sql-data-types.html#standard-types).

### Timestamp values

Druid stores timestamp values as the number of milliseconds since the Unix epoch.
Primary timestamps are stored in a column named `__time`.
If a dataset doesn't have a timestamp, Druid uses the default value of `1970-01-01 00:00:00`.

Druid time functions perform best when used with the `__time` column.
By default, time functions use the UTC time zone.
For more information about timestamp handling, see [Date and time functions](https://druid.apache.org/docs/latest/querying/sql-scalar.html#date-and-time-functions).

Run the following cell to see Druid SQL `TIME_IN_INTERVAL` scalar function at work. This query checks whether a timestamp is contained within a specified interval. The results are grouped by date.

In [ ]:

endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT TIME_IN_INTERVAL(__time, '2016-06-27T12:00:00/2016-06-27T13:00:00'), COUNT(*) FROM \"wikipedia-sql-tutorial\" \
  GROUP BY TIME_IN_INTERVAL(__time, '2016-06-27T12:00:00/2016-06-27T13:00:00')"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

### NULL values

Druid supports SQL compatible NULL handling, allowing string columns to distinguish empty strings from NULL and numeric columns to contain NULL rows. To store and query data in SQL compatible mode, explicitly set the `useDefaultValueForNull` property to `false` in `_common/common.runtime.properties`. See [Configuration reference](https://druid.apache.org/docs/latest/configuration/index.html) for common configuration properties.

When `useDefaultValueForNull` is set to `true` (default behavior), Druid stores NULL values as `0` for numeric columns and as `''` for string columns.

## SELECT statement syntax

Druid SQL supports SELECT statements with the following structure:

``` mysql
[ EXPLAIN PLAN FOR ]
[ WITH tableName [ ( column1, column2, ... ) ] AS ( query ) ]
SELECT [ ALL | DISTINCT ] { * | exprs }
FROM { <table> | (<subquery>) | <o1> [ INNER | LEFT ] JOIN <o2> ON condition }
[ WHERE expr ]
[ GROUP BY [ exprs | GROUPING SETS ( (exprs), ... ) | ROLLUP (exprs) | CUBE (exprs) ] ]
[ HAVING expr ]
[ ORDER BY expr [ ASC | DESC ], expr [ ASC | DESC ], ... ]
[ LIMIT limit ]
[ OFFSET offset ]
[ UNION ALL <another query> ]
```

As a general rule, use the LIMIT clause with `SELECT *` to limit the number of rows returned. 

## WHERE clause

Druid SQL uses the [SQL WHERE clause](https://druid.apache.org/docs/latest/querying/sql.html#where) of a SELECT statement to fetch data based on a particular condition.

In most cases, specifying a time range when using the WHERE clause improves query performance.
This is because Druid partitions data by time and having a time range allows Druid to skip over unrelated data.

Druid supports range filtering on columns that contain long millisecond values, with the boundaries specified as ISO 8601 time intervals. This is suitable for the `__time` column, long metric columns, and dimensions with values that can be parsed as long milliseconds.
    
For example, the following cell uses a comparison operator on the `__time` field to filter results from a certain time range. The cell limits the results to seven rows for display purposes.

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT * FROM \"wikipedia-sql-tutorial\" WHERE __time >= TIMESTAMP '2015-09-12 23:33:55' LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

### Comparison operators

Druid SQL supports the following comparison operators. You can use these operators in conjunction with the WHERE clause to compare expressions.

- equal to (=)
- greater than(>)
- less than (<)
- greater than or equal (>=)
- less than or equal (<=)
- not equal to( <>)

For example, the next cell returns the first seven records that match the following criteria:
- `cityName` is not an empty string
- `countryIsoCode` value equals to `US`

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT * FROM \"wikipedia-sql-tutorial\" WHERE \"cityName\" <> '' AND \"countryIsoCode\" = 'US' LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

### Logical operators

Druid's handling of logical operators is compatible to SQL with a few exceptions:
- In Druid SQL, if the list contains NULL, the IN operator matches NULL values. This behavior is different from the SQL IN operator, which does not match NULL values.
- You can use the LIKE operator with special characters for basic wildcard searches. Druid SQL supports `%` (matches any number of characters) and `_` (matches any one character).

For a complete list of logical SQL operators supported by Druid SQL, see [Logical operators](https://druid.apache.org/docs/latest/querying/sql-operators.html#logical-operators).

## GROUP BY clause

Druid SQL uses the [SQL GROUP BY](https://druid.apache.org/docs/latest/querying/sql.html#group-by) clause to separate items into groups, where each group is composed of rows with identical values. 
The GROUP BY clause is often used with [aggregation functions](https://druid.apache.org/docs/latest/querying/sql-aggregations.html), such as COUNT or SUM, to produce summary values for each group.

For example, the following cell counts all of the entries separated by the field `channel`. The output is limited to seven rows and has two fields: `channel` and `counts`. For each unique value of `channel`, Druid aggregates all rows having that value, counts the number of entries in the group, and assigns the results to a field called `counts`.

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT channel, COUNT(*) AS counts FROM \"wikipedia-sql-tutorial\" GROUP BY channel LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

You can further define the groups by specifying multiple dimensions.
Druid SQL supports using numbers in GROUP BY and ORDER BY clauses to refer to column positions in the SELECT clause.
Similar to SQL, Druid SQL uses one-based indexing to reference elements in SQL statements.

For example, the next cell aggregates entries grouped by fields `cityName` and `countryName`.
The output has three fields: `cityName`, `countryName`, and `counts`. For each unique combination of `cityName` and `countryName`, Druid aggregates all rows and averages the entries in the group.
The output is limited to seven rows for display purposes.

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT cityName, countryName, COUNT(*) AS counts FROM \"wikipedia-sql-tutorial\" GROUP BY 1, 2 LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

## Query types

Druid uses the [Apache Calcite](https://calcite.apache.org/) data management framework to translate SQL statements into Druid’s native JSON-based queries on the Broker. The queries are then executed by the Druid cluster.
Native queries are low level, designed to be lightweight and complete very quickly.

Druid SQL uses the following native query types:
- Scan
- Timeseries
- TopN
- GroupBy

To get information about how a Druid SQL query is translated into a native query type, add [EXPLAIN PLAN FOR](https://druid.apache.org/docs/latest/querying/sql.html#explain-plan) to the beginning of the query.
Alternatively, you can set up [request logging](https://druid.apache.org/docs/latest/configuration/index.html#request-logging).

### Scan

The Scan native query type returns raw Druid rows in streaming mode.
Druid SQL uses the Scan query type for queries that do not aggregate&mdash;queries that do not have GROUP BY or DISTINCT clauses.

For example, run the next cell to scan the `wikipedia-sql-tutorial` table for comments from Mexico City. Calcite translates this Druid SQL query into the Scan native query type.

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT comment AS \"Entry\" FROM \"wikipedia-sql-tutorial\" WHERE cityName = 'Mexico City' LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

### TopN

The TopN native query type returns a sorted set of results for the values in a given dimension according to some criteria. TopN results are always computed in memory. In some cases, the TopN query type delivers approximate ranking and results. To prevent this, set `useApproximateTopN` to `false`.

Druid SQL uses TopN for queries that meet the following criteria:
- queries that GROUP BY a single expression
- queries that have ORDER BY and LIMIT clauses
- queries that do not contain HAVING
- queries that are not nested

For example, the next cell returns the channels based on the number of events for each one in ascending order. Calcite translates this Druid SQL query into the TopN native query type.

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT channel, count(*) as \"Number of events\" FROM \"wikipedia-sql-tutorial\" GROUP BY channel ORDER BY 2 ASC LIMIT 5"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

### Timeseries

The Timeseries native query type returns an array of JSON objects, where each object represents a value asked for by the Timeseries query.

Druid SQL uses Timeseries for queries that meet the following criteria:
- queries that GROUP BY `FLOOR(__time TO unit)` or `TIME_FLOOR(__time, period)`
- queries that do not contain other grouping expressions
- queries that do not contain HAVING
- queries that are not nested
- queries that either have no ORDER BY clause or an ORDER BY clause that orders by the same expression as present in GROUP BY

For example, Calcite translates the following Druid SQL query into the Timeseries native query type:

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT countryName AS \"Country\", SUM(deleted), SUM(added) FROM \"wikipedia-sql-tutorial\" \
  WHERE countryName = 'France' GROUP BY countryName , FLOOR(__time TO HOUR) LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

### GroupBy

The GroupBy native query type returns an array of JSON objects where each object represents a grouping asked for by the GroupBy query. GroupBy delivers exact results and rankings.

Druid SQL uses GroupBy for aggregations, including nested aggregation queries.

For example, Calcite translates the following Druid SQL query into the GroupBy native query type:

In [ ]:
endpoint = "/druid/v2/sql"
print(druid_host+endpoint)
http_method = "POST"

payload = json.dumps({
  "query":"SELECT countryName AS \"Country\", countryIsoCode AS \"ISO\" FROM \"wikipedia-sql-tutorial\" \
  WHERE channel = '#es.wikipedia' GROUP BY countryName, countryIsoCode LIMIT 7"
})
headers = {'Content-Type': 'application/json'}

response = requests.request(http_method, druid_host+endpoint, headers=headers, data=payload)
print(json.dumps(json.loads(response.text), indent=4))

## Learn more

This tutorial covers the basics of Druid SQL. To learn more about querying datasets using Druid SQL, see the following topics:

- [Druid SQL overview](https://druid.apache.org/docs/latest/querying/sql.html) to learn about Druid SQL syntax.
- [SQL data types](https://druid.apache.org/docs/latest/querying/sql-data-types.html) for information on how SQL data types map to Druid SQL.
- [SQL query translation](https://druid.apache.org/docs/latest/querying/sql-translation.html) for best practices that help you minimize the impact of SQL translation.
- [Druid SQL operators](https://druid.apache.org/docs/latest/querying/sql-operators.html) for operators supported by Druid SQL.
- [SQL aggregation functions](https://druid.apache.org/docs/latest/querying/sql-aggregations.html) for reference on the aggregation functions supported by Druid SQL. 
- [Unsupported features](https://druid.apache.org/docs/latest/querying/sql-translation.html#unsupported-features) for a list of SQL features not supported by Druid SQL.